In [1]:
!pip install pandas
!pip install requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
Canada_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text 

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
! pip install bs4
!pip install lxml
!pip install html5lib


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=3b3a7bd8706314a25879752ee1b36757009fdd8b2ee55c0cfd2ac390ed233a32
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(Canada_data, 'html5lib')

In [4]:
tables = soup.find_all('table')
len(tables)
for index,table in enumerate(tables):
 table_index = index
print(tables[table_index].prettify())

<table cellspacing="0" style="background-color: #F8F8F8;" width="100%">
 <tbody>
  <tr>
   <td style="text-align:center; border:1px solid #aaa;">
    <a href="/wiki/Newfoundland_and_Labrador" title="Newfoundland and Labrador">
     NL
    </a>
   </td>
   <td style="text-align:center; border:1px solid #aaa;">
    <a href="/wiki/Nova_Scotia" title="Nova Scotia">
     NS
    </a>
   </td>
   <td style="text-align:center; border:1px solid #aaa;">
    <a href="/wiki/Prince_Edward_Island" title="Prince Edward Island">
     PE
    </a>
   </td>
   <td style="text-align:center; border:1px solid #aaa;">
    <a href="/wiki/New_Brunswick" title="New Brunswick">
     NB
    </a>
   </td>
   <td colspan="3" style="text-align:center; border:1px solid #aaa;">
    <a href="/wiki/Quebec" title="Quebec">
     QC
    </a>
   </td>
   <td colspan="5" style="text-align:center; border:1px solid #aaa;">
    <a href="/wiki/Ontario" title="Ontario">
     ON
    </a>
   </td>
   <td style="text-align:center; b

In [5]:
column_names = ['PostalCode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

In [6]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


In [7]:
# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [8]:
df = df.dropna()
empty = 'Not assigned'
df = df[(df.PostalCode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [9]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [10]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['PostalCode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [11]:
print(df2.shape)
df2.head(10)

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [12]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_lon_lat = pd.read_csv('Toronto_long_lat_data.csv')
df_lon_lat.columns=['PostalCode','Latitude','Longitude']
df_lon_lat.head(12)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [13]:
Toronto_df = pd.merge(df2,df_lon_lat, on='PostalCode')
Toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [14]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.13.0             |     pyhd8ed1ab_0          97 KB  conda-forge
    aiohttp-3.7.4.post0        |   py37h5e8e339_0         625 KB  conda-forge
    alsa-lib-1.2.3             |       h516909a_0         560 KB  conda-forge
   

In [15]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:

map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
# defining radius and limit of venues to get
radius=500
LIMIT=100

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'SS2MEJUIINVDIJQBQSOVLD4F0GASSZCEKMYI4J35V4H45BQE', 
            'HMCUCAXXWML5R3KB3DQOWZCGOIRR4LL2QA2P0GPYYVS2RHIL', 
            '20180604', 
            43.6534817, 
            -79.3839347, 
            500, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
 
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
toronto_venues = getNearbyVenues(names=Toronto_df['Neighborhood'],
                                   latitudes=Toronto_df['Latitude'],
                                   longitudes=Toronto_df['Longitude']
                                )

toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,"Malvern, Rouge",43.806686,-79.194353,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,"Malvern, Rouge",43.806686,-79.194353,LUSH,43.653557,-79.380400,Cosmetics Shop
3,"Malvern, Rouge",43.806686,-79.194353,M Square Coffee Co,43.651218,-79.383555,Coffee Shop
4,"Malvern, Rouge",43.806686,-79.194353,Indigo,43.653515,-79.380696,Bookstore
5,"Malvern, Rouge",43.806686,-79.194353,Japango,43.655268,-79.385165,Sushi Restaurant
6,"Malvern, Rouge",43.806686,-79.194353,Magic Tailor,43.653742,-79.379745,Clothing Store
7,"Malvern, Rouge",43.806686,-79.194353,Kathy's Sushi and Bento,43.655031,-79.386724,Sushi Restaurant
8,"Malvern, Rouge",43.806686,-79.194353,John & Sons Oyster House,43.650656,-79.381613,Seafood Restaurant
9,"Malvern, Rouge",43.806686,-79.194353,The Keg Steakhouse + Bar - York Street,43.649987,-79.384103,Restaurant


 Size of the resulting dataframe

In [25]:
toronto_venues.shape

(6592, 7)

How many venues are returned for each neighborhood?

In [28]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,64,64,64,64,64,64
"Alderwood, Long Branch",64,64,64,64,64,64
"Bathurst Manor, Wilson Heights, Downsview North",64,64,64,64,64,64
Bayview Village,64,64,64,64,64,64
"Bedford Park, Lawrence Manor East",64,64,64,64,64,64
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",64,64,64,64,64,64
Woburn,64,64,64,64,64,64
Woodbine Heights,64,64,64,64,64,64


Analysing Each Neighborhood

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

,American Restaurant,Asian Restaurant,Bakery,Bank,Bar,Bookstore,Bubble Tea Shop,Café,Clothing Store,Cocktail Bar,...,Sandwich Place,Seafood Restaurant,Shoe Store,Steakhouse,Sushi Restaurant,Tea Room,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Dataframe size
toronto_onehot.shape

(6592, 43)

group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [36]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Asian Restaurant,Bakery,Bank,Bar,Bookstore,Bubble Tea Shop,Café,Clothing Store,...,Sandwich Place,Seafood Restaurant,Shoe Store,Steakhouse,Sushi Restaurant,Tea Room,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,Agincourt,0.015625,0.015625,0.03125,0.015625,0.015625,0.015625,0.015625,0.015625,0.09375,...,0.03125,0.015625,0.015625,0.015625,0.046875,0.015625,0.03125,0.015625,0.015625,0.015625
1,"Alderwood, Long Branch",0.015625,0.015625,0.03125,0.015625,0.015625,0.015625,0.015625,0.015625,0.09375,...,0.03125,0.015625,0.015625,0.015625,0.046875,0.015625,0.03125,0.015625,0.015625,0.015625
2,"Bathurst Manor, Wilson Heights, Downsview North",0.015625,0.015625,0.03125,0.015625,0.015625,0.015625,0.015625,0.015625,0.09375,...,0.03125,0.015625,0.015625,0.015625,0.046875,0.015625,0.03125,0.015625,0.015625,0.015625
3,Bayview Village,0.015625,0.015625,0.03125,0.015625,0.015625,0.015625,0.015625,0.015625,0.09375,...,0.03125,0.015625,0.015625,0.015625,0.046875,0.015625,0.03125,0.015625,0.015625,0.015625
4,"Bedford Park, Lawrence Manor East",0.015625,0.015625,0.03125,0.015625,0.015625,0.015625,0.015625,0.015625,0.09375,...,0.03125,0.015625,0.015625,0.015625,0.046875,0.015625,0.03125,0.015625,0.015625,0.015625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,"Willowdale, Newtonbrook",0.015625,0.015625,0.03125,0.015625,0.015625,0.015625,0.015625,0.015625,0.09375,...,0.03125,0.015625,0.015625,0.015625,0.046875,0.015625,0.03125,0.015625,0.015625,0.015625
99,Woburn,0.015625,0.015625,0.03125,0.015625,0.015625,0.015625,0.015625,0.015625,0.09375,...,0.03125,0.015625,0.015625,0.015625,0.046875,0.015625,0.03125,0.015625,0.015625,0.015625
100,Woodbine Heights,0.015625,0.015625,0.03125,0.015625,0.015625,0.015625,0.015625,0.015625,0.09375,...,0.03125,0.015625,0.015625,0.015625,0.046875,0.015625,0.03125,0.015625,0.015625,0.015625
101,York Mills West,0.015625,0.015625,0.03125,0.015625,0.015625,0.015625,0.015625,0.015625,0.09375,...,0.03125,0.015625,0.015625,0.015625,0.046875,0.015625,0.03125,0.015625,0.015625,0.015625


print each neighborhood along with the top 5 most common venues

In [37]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0       Clothing Store  0.09
1          Coffee Shop  0.09
2  Japanese Restaurant  0.05
3     Sushi Restaurant  0.05
4           Restaurant  0.03


----Alderwood, Long Branch----
                 venue  freq
0       Clothing Store  0.09
1          Coffee Shop  0.09
2  Japanese Restaurant  0.05
3     Sushi Restaurant  0.05
4           Restaurant  0.03


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0       Clothing Store  0.09
1          Coffee Shop  0.09
2  Japanese Restaurant  0.05
3     Sushi Restaurant  0.05
4           Restaurant  0.03


----Bayview Village----
                 venue  freq
0       Clothing Store  0.09
1          Coffee Shop  0.09
2  Japanese Restaurant  0.05
3     Sushi Restaurant  0.05
4           Restaurant  0.03


----Bedford Park, Lawrence Manor East----
                 venue  freq
0       Clothing Store  0.09
1          Coffee Shop  0.09
2  Japanese Restaurant  0.05
3     Sus

Cluster neighborhoods

In [42]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 
# to change use .astype()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:7: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [44]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

toronto_merged = Toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
toronto_merged=toronto_merged.dropna()
toronto_merged['Cluster_Labels'] = toronto_merged.Cluster_Labels.astype(int)

# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters